## Train a simple convnet on the Fashion MNIST dataset

In this, we will see how to deal with image data and train a convnet for image classification task.

### Load the  `fashion_mnist`  dataset

** Use keras.datasets to load the dataset **

In [ ]:
import pandas as pd
import numpy as np
import keras
from keras.utils import np_utils

In [ ]:
from keras.datasets import fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()


### Find no.of samples are there in training and test datasets

In [ ]:

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


In [ ]:
print(x_train.shape[0])
print(x_test.shape[0])
print(len(x_test))

### Find dimensions of an image in the dataset

In [ ]:
x_train[0].shape

### Convert train and test labels to one hot vectors

** check `keras.utils.to_categorical()` **

In [ ]:
# convert integers to dummy variables (i.e. one hot encoded)
dummys_ytrain = np_utils.to_categorical(y_train)
print(dummys_ytrain)
dummys_ytrain = np_utils.to_categorical(y_test)
print(dummys_ytrain)

### Normalize both the train and test image data from 0-255 to 0-1

In [ ]:
x_trainN = x_train/ 255
x_testN = x_test/ 255
print(x_train[0])
print(x_trainN[0])

### Reshape the data from 28x28 to 28x28x1 to match input dimensions in Conv2D layer in keras

In [ ]:

x_trainNR = x_trainN.reshape((60000, 28, 28,1))
x_testNR = x_testN.reshape((10000,28,28, 1))

### Import the necessary layers from keras to build the model

In [ ]:
from keras.datasets import cifar10, mnist
from keras.models import Sequential# sequential is one type of model; there are graph models as well
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape # Dense is fully connected layer
from keras.layers import Convolution2D, MaxPooling2D #
from keras.utils import np_utils
import pickle # serialization fancy word for storing on disk
from matplotlib import pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (15, 8)

### Build a model 

** with 2 Conv layers having `32 3*3 filters` in both convolutions with `relu activations` and `flatten` before passing the feature map into 2 fully connected layers (or Dense Layers) having 128 and 10 neurons with `relu` and `softmax` activations respectively. Now, using `categorical_crossentropy` loss with `adam` optimizer train the model with early stopping `patience=5` and no.of `epochs=10`. **

In [ ]:
TRAIN = False
BATCH_SIZE = 32
EPOCHS = 10 

# Define the Type of Model
model2 = Sequential()
# 1st Conv Layer
model2.add(Convolution2D(32, 3, 3, input_shape=(28, 28, 1)))
model2.add(Activation('relu'))

# 2nd Conv Layer
model2.add(Convolution2D(32, 3, 3))# input shape is known in the previous layer. so we don't give input shape
model2.add(Activation('relu'))

# Fully Connected Layer
model2.add(Flatten())
model2.add(Dense(128))
model2.add(Activation('relu'))


# Layer 1
model2.add(Dense(output_dim=128, init='he_normal', bias=True))
model2.add(Activation("relu"))

# Layer 2
model2.add(Dense(output_dim=10, init='he_normal', bias=True))
model2.add(Activation("softmax"))

# Loss and Optimizer
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Store Training Results
early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
callback_list = [early_stopping]# [stats, early_stopping]

# Train the model
#model2.fit(x_trainNR, dummys_ytrain, nb_epoch=EPOCHS, batch_size=BATCH_SIZE,
        #validation_data=(x_testNR, dummy_ytest), callbacks=callback_list, verbose=True)

### Now, to the above model add `max` pooling layer of `filter size 2x2` and `dropout` layer with `p=0.25` after the 2 conv layers and run the model

In [ ]:
TRAIN = False
BATCH_SIZE = 32
EPOCHS = 10 

# Define the Type of Model
model3 = Sequential()
# 1st Conv Layer
model3.add(Convolution2D(32, 3, 3, input_shape=(28, 28, 1)))
model3.add(Activation('relu'))

# 2nd Conv Layer
model3.add(Convolution2D(32, 3, 3))# input shape is known in the previous layer. so we don't give input shape
model3.add(Activation('relu'))

# Max Pooling
model3.add(MaxPooling2D(pool_size=(2,2)))
# Dropout
model3.add(Dropout(0.25)) #drop-out with p = 0.25

# Fully Connected Layer
model3.add(Flatten())
model3.add(Dense(128))
model3.add(Activation('relu'))


# Layer 1
model3.add(Dense(output_dim=128, init='he_normal', bias=True))
model3.add(Activation("relu"))

# Layer 2
model3.add(Dense(output_dim=10, init='he_normal', bias=True))
model3.add(Activation("softmax"))

# Loss and Optimizer
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Store Training Results
early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
callback_list = [early_stopping]# [stats, early_stopping]

# Train the model
#model3.fit(x_trainNR, dummys_ytrain, nb_epoch=EPOCHS, batch_size=BATCH_SIZE,
#  validation_data=(x_testNR, dummy_ytest), callbacks=callback_list, verbose=True)

### Now, to the above model, lets add Data Augmentation 

### Import the ImageDataGenrator from keras and fit the training images

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=50,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images

# Prepare the generator
datagen.fit(x_trainNR)

#### Showing 5 versions of the first image in training dataset using image datagenerator.flow()

In [ ]:
from matplotlib import pyplot as plt
gen = datagen.flow(x_trainNR[0:1], batch_size=1)
for i in range(1, 6):
    plt.subplot(1,5,i)
    plt.axis("off")
    plt.imshow(gen.next().squeeze(), cmap='gray')
    plt.plot()
plt.show()

### Run the above model using fit_generator()

In [ ]:
#H = model3.fit_generator(datagen.flow(x_trainNR, dummys_ytrain, batch_size=32),
#	validation_data=(x_testNR, dummy_ytest), steps_per_epoch=len(x_trainNR) /32,
#	epochs=EPOCHS)

###  Report the final train and validation accuracy

In [ ]:
model3.evaluate(x_trainNR,dummys_ytrain)

## **DATA AUGMENTATION ON CIFAR10 DATASET**

One of the best ways to improve the performance of a Deep Learning model is to add more data to the training set. Aside from gathering more instances from the wild that are representative of the distinction task, we want to develop a set of methods that enhance the data we already have. There are many ways to augment existing datasets and produce more robust models. In the image domain, these are done to utilize the full power of the convolutional neural network, which is able to capture translational invariance. This translational invariance is what makes image recognition such a difficult task in the first place. You want the dataset to be representative of the many different positions, angles, lightings, and miscellaneous distortions that are of interest to the vision task.

### **Import neessary libraries for data augmentation**

### **Load CIFAR10 dataset**

In [ ]:
from keras.datasets import cifar10
(x_train1, y_train1), (x_test1, y_test1) = cifar10.load_data()


### **Create a data_gen funtion to genererator with image rotation,shifting image horizontally and vertically with random flip horizontally.**

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=50,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images

# Prepare the generator
datagen.fit(x_train1)

### **Prepare/fit the generator.**

In [ ]:
from matplotlib import pyplot as plt
from scipy.misc import toimage
gen = datagen.flow(x_train1[7:8], batch_size=1)
for i in range(1, 6):
    plt.subplot(1,5,i)
    plt.axis("off")
    plt.imshow(toimage(gen.next().squeeze()), cmap='gray')
    plt.plot()
plt.show()

### **Generate 5 images for 1 of the image of CIFAR10 train dataset.**